Scraper
---

Author: Peter Zhang

Scraping tool for Tabroom and the wiki.

## Setup

#### Imports

In [167]:
# imports
import urllib.request, urllib.parse, urllib.error
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import csv
import os.path
from os import path
import re

#### Settings

- OVERWRITE determines whether or not to update existing files.
- PAGES_URL is a list of Wiki pages

In [2]:
# settings
OVERWRITE = False

In [30]:
# get page URLs
WIKIS_URL = "tools/wiki_pages.csv"
WIKIS = [row for row in csv.reader(open(WIKIS_URL, 'r'))]

In [33]:
# outpath
OUTPATH = "wiki_path/"

### Scrapers

#### Wiki Page

Take the URL to a wiki page and returns a list of all the school names and their respective URLs.

In [151]:
# for a given archive year, return all schools
def getSchools(url):
    
    # try opening URL
    try:
        html = urlopen(url).read()
    except Exeption:
        print("Wiki URL broke")
        return []
    
    
    # soupify
    soup = BeautifulSoup(html, "html.parser")
    
    # find all links
    links = soup.find_all('a')
    
    # find only school links
    links = [link for link in links if "(" in link.text and ")" in link.text]
    
    # get link text and URL
    schools = [[link.text, link.get('href')] for link in links]

    return schools

In [152]:
getSchools("https://hspolicy19.debatecoaches.org/")

[['Advanced Technologies Academy (NV)', '/Advanced%20Technologies%20Academy/'],
 ['Airline (AL)', '/Airline/'],
 ['Aledo (TX)', '/Aledo/'],
 ['Alief Taylor (TX)', '/Alief%20Taylor/'],
 ['Alpharetta (GA)', '/Alpharetta/'],
 ['Altamont (AL)', '/Altamont/'],
 ['American Heritage (FL)', '/American%20Heritage/'],
 ['Anderson (TX)', '/Anderson/'],
 ['Andover (KS)', '/Andover/'],
 ['Andover Central (KS)', '/Andover%20Central/'],
 ['Andrews (TX)', '/Andrews/'],
 ['Archbishop Mitty (CA)', '/Archbishop%20Mitty/'],
 ['Ashland (OR)', '/Ashland/'],
 ['Asian Debate League (AK)', '/Asian%20Debate%20League/'],
 ['Athens (TX)', '/Athens/'],
 ['Aubrey (TX)', '/Aubrey/'],
 ['BASIS Chandler (AZ)', '/BASIS%20Chandler/'],
 ['BASIS Peoria (AZ)', '/BASIS%20Peoria/'],
 ['BASIS Shavano (TX)', '/BASIS%20Shavano/'],
 ['Bakersfield (CA)', '/Bakersfield/'],
 ['Baltimore City College (MD)', '/Baltimore%20City%20College/'],
 ['Barstow (MO)', '/Barstow/'],
 ['Baton Rouge Magnet (LA)', '/Baton%20Rouge%20Magnet/'],
 ['B

#### School Page

Take the URL to a school and return a list of debaters and the URLs that point to their Aff and Neg pages

In [153]:
# return aff and neg URLs for all the debaters of a given school

def getTeams(url):
    
    # try opening URL
    try:
        html = urlopen(url).read()
    except Exception:
        return (0, [], {}, "School URL broke")
        
    # soupify
    soup = BeautifulSoup(html, "html.parser")
    
    # look for a table
    if len(soup.find_all("table")):
        table = soup.find_all("table")[0]
    else:
        return (0, [], {}, "School page not set up")
        
    # exclude header
    numDebaters = len(table.find_all('tr'))-1
    
    if numDebaters == 0:
        return (0, [], {}, "No debaters listed")
    
    # track team names and map to URLs
    teamNames = []
    pageURLs = {}
    
    
    # for each table row
    for row in table.find_all('tr')[1:]:
        
        # the first column's string is the team name
        team = row.find('td').string
        teamNames.append(team)
        
        # collect the URLs to the aff and neg page
        links = row.find_all('a')
        URLs = [link.get("href") for link in links]
        pageURLs[team] = URLs
    
    return (numDebaters, teamNames, pageURLs, "")


In [35]:
getTeams("https://hspolicy19.debatecoaches.org/Advanced%20Technologies%20Academy")

(3,
 ['Advanced Technologies Academy Jacqueline Balanovsky - Raymond Behnke',
  'Advanced Technologies Academy Sam Self - Jonah Gentleman',
  'Advanced Technologies Academy Ryan Fritchel - Hannah Lewis'],
 {'Advanced Technologies Academy Jacqueline Balanovsky - Raymond Behnke': ['/Advanced%20Technologies%20Academy/Balanovsky-Behnke%20Aff',
   '/Advanced%20Technologies%20Academy/Balanovsky-Behnke%20Neg'],
  'Advanced Technologies Academy Sam Self - Jonah Gentleman': ['/Advanced%20Technologies%20Academy/Self-Gentleman%20Aff',
   '/Advanced%20Technologies%20Academy/Self-Gentleman%20Neg'],
  'Advanced Technologies Academy Ryan Fritchel - Hannah Lewis': ['/Advanced%20Technologies%20Academy/Fritchel-Lewis%20Aff',
   '/Advanced%20Technologies%20Academy/Fritchel-Lewis%20Neg']})

#### Team Page

In [187]:
def getDisclosure(url):

    # try opening URL
    try:
        html = urlopen(url).read().decode('utf-8-sig')
        
    except Exception:
        return {"Team URL":url,
                "Rounds": [],
                "Round Reports": [],
                "Cites": [],
                "Errors": "Did not open"}
    
    
    # soupify
    soup = BeautifulSoup(html, "html.parser")
    
    tables = soup.find_all("table")

    if len(tables) > 2:
        
        rnds = tables[0]
        rndrprts = tables[1]
        cts = tables[2]
    
    else:
        
        return {"Team URL":url,
                "Rounds": [],
                "Round Reports": [],
                "Cites": [],
                "Errors": "Not configured properly"}
    
    # collect all rounds
    rounds = []
    
    for row in rnds.find_all("tr")[1:]:
        
        # get pieces
        cols = row.find_all("td")
        trn = cols[0].text
        rnd = cols[1].text
        opp = cols[2].text
        jdg = cols[3].text
        osrc = cols[6].find("a")
        if osrc:
            docURL = osrc.get("href")
        else:
            docURL = ""
            
        rounds.append([trn, rnd, opp, jdg, docURL])
    
    # collect all round reports
    roundReports = []
    
    for row in rndrprts.find_all("tr")[1:]:
        roundReports.append(row.find_all("td")[2].find_all("p")[1].text)

    # collect all cites
    cites = []
    
    for row in cts.find_all("tr")[1:]:
    
        uploadDate = row.find_all("td")[1]
        text = " ".join([par.text for par in row.find_all("td")[0].find_all("p")[1:]])
        text = re.sub(r'([^\s\w]|_)+', '', text)
        cites.append(uploadDate.text + text)
        
    return {"Team URL":url,
            "Rounds": rounds,
            "Round Reports": roundReports,
            "Cites": " ".join(cites),
            "Errors": ""}
    


In [185]:
getDisclosure("https://hsld17.debatecoaches.org/Acton-Boxborough/Liu%20Aff")

{'Team URL': 'https://hsld17.debatecoaches.org/Acton-Boxborough/Liu%20Aff',
 'Rounds': [['-', 'Finals', '-', '-', ''],
  ['Harvard', '2', 'Dan Shahab', 'David Moon', ''],
  ['Lexington Winter Invitational', '2', 'Scarsdale AW', 'David Moon', ''],
  ['Newark Invitational', '4', 'Stuyvesant JL', 'Amit Kukreja', ''],
  ['Practice Round', '1', 'Michelle I drop T Li', 'the coolest judge', ''],
  ['Practice Round', 'Semis', '-', '-', '']],
 'Round Reports': [],
 'Cites': ['2/13/18Tournament   Round Finals  Opponent   Judge Email 20liumabschoolsorgFacebook Matthew Liu httpswwwfacebookcommatthewliu7355Messenger is the easiest way to contact meI also reserve the right to read positions on the following peoples wikisLexington PSLexington KYHunter MLNeedham ZLMillburn KWAny of my teammates',
  '1/15/18Tournament Newark Invitational  Round 4  Opponent Stuyvesant JL  Judge Amit Kukreja Winter and Leighton 01\xa0Winter Deborah Dunann and Dana C Leighton Structural Violence Section Introduction sites

### Scraping


In [188]:
for wiki in WIKIS:
        
    wikiName = wiki[0]
    wikiURL = wiki[1]
    
    # if it exists, don't overwrite
    if (not OVERWRITE) and path.exists(OUTPATH + wikiName + "_schools.csv"):
        continue
        
    # open output file
    with open("wiki_data/" + wikiName + "schools_wiki.csv", 'w', encoding="utf-8") as SCHOOLFILE, open("wiki_data/" + wikiName + "teams_wiki.csv", 'w', encoding="utf-8") as TEAMFILE:
        
        # instantaite writer
        schoolWriter = csv.DictWriter(SCHOOLFILE,
                                     fieldnames = ["School Name",
                                                   "School URL",
                                                   "School Teams",
                                                   "Errors"],
                                      quotechar='"', 
                                      quoting=csv.QUOTE_NONNUMERIC,
                                     lineterminator = "\n")
        
        # write header
        schoolWriter.writeheader()
        
        # instantaite writer
        teamWriter = csv.DictWriter(TEAMFILE,
                                    fieldnames = ["Team Name",
                                                  "Side",
                                                  "Team URL",
                                                  "Rounds",
                                                  "Round Reports",
                                                  "Cites",
                                                  "Errors"],
                                    lineterminator = "\n")
        
        # write header
        teamWriter.writeheader()
            
        print("Scraping " + wikiName)
        
        # get school URLs
        schools = getSchools(wikiURL)
        
        print("There are " + str(len(schools)) + " schools")
        
        for school in schools:
            
            # get school name and URL
            schoolName, schoolURL = school[0], wikiURL + school[1]
            
            print("Checking " + schoolName)
            
            # get teams from the school
            numTeams, teamNames, teamURLs, errors = getTeams(schoolURL)
            
            print("Found " + str(numTeams) + " teams")
            
            # write school information
            schoolWriter.writerow({"School Name" : schoolName,
                                  "School URL": schoolURL,
                                  "School Teams": teamNames,
                                  "Errors": errors})
            
            for team in teamNames:
                
                print("Scraping " + team)
                
                # get team URL
                teamURL = teamURLs[team]
                affURL = wikiURL + teamURL[0]
                negURL = wikiURL + teamURL[1]
                
                # aff scraping
                affInfo = getDisclosure(affURL)
                affInfo["Team Name"] = team
                affInfo["Side"] = "Aff"
                    
                # write aff info
                teamWriter.writerow(affInfo)

                # neg scraping
                negInfo = getDisclosure(negURL)
                negInfo["Team Name"] = team
                negInfo["Side"] = "Neg"
                    
                # write neg info
                teamWriter.writerow(negInfo)

Scraping LD17
There are 495 schools
Checking Aberdeen Central (SD)
Found 0 teams
Checking Academy of Higher Learning (CA)
Found 0 teams
Checking Acton-Boxborough (MA)
Found 3 teams
Scraping Acton-Boxborough - Matthew Liu
Scraping Acton-Boxborough - Jeffrey Huang


KeyboardInterrupt: 